<a href="https://colab.research.google.com/github/valesierrai/Introduccion-a-la-Inteligencia-de-Negocios/blob/main/Reto2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**0. Cargamos Librerias**

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

#**1. Cargamos la Base de Datos**

In [ ]:
datos = pd.read_excel('/content/Tablas Reto 2.xlsx',
                      sheet_name= None, header= 0 )

In [ ]:
#Miramos las tablas dentro de la base de datos
datos.keys()

dict_keys(['InfoCompañia', 'InfoFinanciera'])

#**2. Cargamos SQLite**

In [ ]:
#Se crea la base de datos
conexion = sqlite3.connect('ConsultaComercioElectronico.db')
cursor = conexion.cursor()

In [ ]:
#Dado que se tienen varias tablas, usaremos un ciclo para pasar la información desde el diccionario hasta la base de datos creada
for hoja, datos in datos.items():
  datos.to_sql(hoja, conexion, if_exists = 'replace', index = False)

In [ ]:
#Confirmamos los cambios ejecutados en la base de datos
conexion.commit()

#**3. Creamos las consultas**

In [ ]:
#1. ¿Cuántas compañias hay por cada país?
consulta = '''SELECT País,
             COUNT (*) AS cantidad
             FROM InfoCompañia
             GROUP BY País'''

#Ejecutamos la consulta
cursor.execute(consulta)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Argentina', 33)
('Brasil', 11)
('Chile', 416)
('Colombia', 9540)


In [ ]:
#Mostramos la tabla resultante en pandas
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta = pd.DataFrame(resultados, columns = columnas)
df_consulta

,País,cantidad
0,Argentina,33
1,Brasil,11
2,Chile,416
3,Colombia,9540


In [ ]:
#2. ¿Cuáles compañias están en las ciudades principales de Colombia (Bogotá, Medellín, Cali, Barranquilla, Santa Marta, Cartagena) en orden descendente?
consulta1 = '''SELECT Ciudad,
             Compañía,
             COUNT (*) AS cantidad
             FROM InfoCompañia
             WHERE País = 'Colombia' AND Ciudad IN ('Bogotá D.C.', 'Medellín', 'Cali', 'Barranquilla', 'Santa Marta', 'Cartagena de Indias')
             GROUP BY Ciudad
             ORDER BY cantidad DESC'''

#Ejecutamos la consulta
cursor.execute(consulta1)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Bogotá D.C.', 'Frontera Energy Colombia Corp., Sucursal Colombia', 4257)
('Medellín', 'Novaventa S.A.S.', 1055)
('Cali', 'Pallomaro S.A.', 878)
('Barranquilla', 'Soluingeniar Curtidos y Pieles S.A.S.', 420)
('Cartagena de Indias', 'Grupo Empresarial Lupa S.A.S.', 171)
('Santa Marta', 'Inversiones Castro & Calderon S.A.S.', 50)


In [ ]:
#Mostramos la tabla resultante en pandas
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta1 = pd.DataFrame(resultados, columns = columnas)
df_consulta1

,Ciudad,Compañía,cantidad
0,Bogotá D.C.,"Frontera Energy Colombia Corp., Sucursal Colombia",4257
1,Medellín,Novaventa S.A.S.,1055
2,Cali,Pallomaro S.A.,878
3,Barranquilla,Soluingeniar Curtidos y Pieles S.A.S.,420
4,Cartagena de Indias,Grupo Empresarial Lupa S.A.S.,171
5,Santa Marta,Inversiones Castro & Calderon S.A.S.,50


In [ ]:
#3. ¿Cuáles son las compañias que tienen ingresos operativos de más de 1000 millones de dólares?
consulta2 = '''SELECT Compañía,
             [Total Ingreso Operativo] AS Ingreso_Operativo
             FROM InfoFinanciera
             WHERE Ingreso_Operativo > 1000'''

#Ejecutamos la consulta
cursor.execute(consulta2)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Claro S.A. (Sao Paulo) CNPJ (40432544/0001-47)', 9162.06)
('Magazine Luiza S.A.', 7246.82)
('Americanas S.A - em Recuperacao Judicial', 4998.0)
('Martins Comercio e Servicos de Distribuicao S.A.', 1105.22)
('Frontera Energy Colombia Corp., Sucursal Colombia', 1096.31)


In [ ]:
#Mostramos la tabla resultante en pandas
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta2 = pd.DataFrame(resultados, columns = columnas)
df_consulta2

,Compañía,Ingreso_Operativo
0,Claro S.A. (Sao Paulo) CNPJ (40432544/0001-47),9162.06
1,Magazine Luiza S.A.,7246.82
2,Americanas S.A - em Recuperacao Judicial,4998.00
3,Martins Comercio e Servicos de Distribuicao S.A.,1105.22
4,"Frontera Energy Colombia Corp., Sucursal Colombia",1096.31


In [ ]:
#4. Con base a la información anterior, ¿A que país y ciudad pertencen estas compañías?
consulta3 = '''SELECT IC.Compañía,
             IC.País,
             IC.Ciudad,
             InfoFinanciera.[Total Ingreso Operativo] AS Ingreso_Operativo
             FROM InfoFinanciera
             JOIN `InfoCompañia` IC ON InfoFinanciera.Compañía = IC.Compañía
             WHERE InfoFinanciera.[Total Ingreso Operativo] > 1000''' # Use the actual table name to reference the column

#Ejecutamos la consulta
cursor.execute(consulta3)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Claro S.A. (Sao Paulo) CNPJ (40432544/0001-47)', 'Brasil', 'Sao Paulo', 9162.06)
('Magazine Luiza S.A.', 'Brasil', 'Franca', 7246.82)
('Americanas S.A - em Recuperacao Judicial', 'Brasil', 'Rio de Janeiro', 4998.0)
('Martins Comercio e Servicos de Distribuicao S.A.', 'Brasil', 'Uberlandia', 1105.22)
('Frontera Energy Colombia Corp., Sucursal Colombia', 'Colombia', 'Bogotá D.C.', 1096.31)


In [ ]:
#Mostramos la tabla resultante en pandas
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta3 = pd.DataFrame(resultados, columns = columnas)
df_consulta3

,Compañía,País,Ciudad,Ingreso_Operativo
0,Claro S.A. (Sao Paulo) CNPJ (40432544/0001-47),Brasil,Sao Paulo,9162.06
1,Magazine Luiza S.A.,Brasil,Franca,7246.82
2,Americanas S.A - em Recuperacao Judicial,Brasil,Rio de Janeiro,4998.00
3,Martins Comercio e Servicos de Distribuicao S.A.,Brasil,Uberlandia,1105.22
4,"Frontera Energy Colombia Corp., Sucursal Colombia",Colombia,Bogotá D.C.,1096.31


In [ ]:
#5. ¿Cuáles empresesa tienen una tendencia de los ingresos operacionales que tengan un crecimiento de más de 100%, que sean de Chile y Argentina, y a que sector pertenece?
consulta4 = '''SELECT IC.Compañía,
             IC.País,
             IC.[Sector (EMIS Industries)],
             InfoFinanciera.[Tendencia De Los Ingresos Operacionales (%)] AS Tendencia
             FROM InfoFinanciera
             JOIN `InfoCompañia` IC ON InfoFinanciera.Compañía = IC.Compañía
             WHERE Tendencia > 100 AND País IN ('Chile', 'Argentina') '''

#Ejecutamos la consulta
cursor.execute(consulta4)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Delivery Hero Stores S.A.S.', 'Argentina', 'Bienes y servicios de consumo(12); Comercio minorista(12.1); Comercio electrónico(12.1.6); Servicios(17); Servicios de apoyo a las empresas(17.2); Servicios de marketing y publicidad(17.2.1)', 186.98)


In [ ]:
#6. ¿Cuántas compañias tienen su proceso auditado?
consulta5 = '''SELECT
             COUNT (*) AS cantidad
             FROM InfoFinanciera
             WHERE Auditado = 'Auditado' '''

#Ejecutamos la consulta
cursor.execute(consulta5)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

(113,)


In [ ]:
#7. Con base a la información anterior, dividimos la información en países
consulta6 = '''SELECT País,
             COUNT (*) AS cantidad
             FROM InfoFinanciera
             JOIN `InfoCompañia` IC ON InfoFinanciera.Compañía = IC.Compañía
             WHERE Auditado = 'Auditado' '''

#Ejecutamos la consulta
cursor.execute(consulta6)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Brasil', 113)


In [ ]:
#8. ¿Cuantas compañías están consolidadas y cuántas están individuales?
consulta7 = '''SELECT Consolidado,
             COUNT (*) AS cantidad
             FROM InfoFinanciera
             GROUP BY Consolidado'''

#Ejecutamos la consulta
cursor.execute(consulta7)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

(None, 8778)
('Consolidado', 4)
('Individual', 1213)


In [ ]:
#Mostramos la tabla resultante en pandas
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta7 = pd.DataFrame(resultados, columns = columnas)
df_consulta7

,Consolidado,cantidad
0,None,8778
1,Consolidado,4
2,Individual,1213


In [ ]:
#Cerramos la conexion
conexion.close()